Purpose: Aggregating NCES Demographic data

The data was taken from IES NCES Education Demographic and Geographic estimates website (https://nces.ed.gov/programs/edge/tableviewer/acsProfile/2022) for each  population group. The total of 15 text files were converted into .xlsx. The data was taken from 2018-2022 (latest).

- Total Populations
- Total Children
- Relevant Children
- Total Parents
- Parents of Relevant Children

Each population group has 3 types of data: 
- Social
- Demographic
- Economic

Each of these type of data contains many columns regarding the details of the population which contains: 
- Estimate
- Margin of error
- Percent estimate 
- Percent margin of error

Each of the columns has a code name, which can be referenced from ACS-ED_2018-2022_RecordLayouts file in each folder. Because of the issue with having the same code name for columns of similar population (Total Children vs Relevant Children, and Total Parents vs Parents of Relevant Children), the code name for columns of Total Children and Total Parents of Relevant Children were added a "T" into the beginning of the code name (example: CDP02_2est becomes TCDP02_2est for Married Couple Household of Total Children file while CDP02_2est is still Married Couple Household for Relevant Children file).

Only the Estimate and Percent Estimate columns were kept after merging. There is one Finance_main_data excel file containing both types of columns (1556 cols, 13399 rows (districts))for all files and 2 excel files containing Estimate (1320 cols, 13399 districts) and Percent Estimate (1126 cols, 13399 districts ) separately. 



In [1]:
# Converting from txt to xlsx

import os
import pandas as pd

# Get the current working directory (where the notebook is located)
current_dir = os.path.abspath("")  # This will get the current working directory

# Function to convert text file to Excel with "|" delimiter
def txt_to_excel(txt_file, excel_file):
    # Read the text file into a DataFrame with "|" delimiter and low_memory=False
    df = pd.read_csv(txt_file, delimiter='|', header=None, low_memory=False)
    # Write the DataFrame to an Excel file
    df.to_excel(excel_file, index=False, header=False)

# Iterate over the folders and files in the current directory
for item_name in os.listdir(current_dir):
    item_path = os.path.join(current_dir, item_name)
    
    # Check if it's a directory and not the .ipynb_checkpoints
    if os.path.isdir(item_path) and item_name != '.ipynb_checkpoints':
        for file_name in os.listdir(item_path):
            file_path = os.path.join(item_path, file_name)
            
            # Check if it's a text file
            if file_name.endswith('.txt'):
                # Define the Excel file path within the same directory as the text file
                excel_file_name = f"{item_name}.xlsx"
                excel_file_path = os.path.join(item_path, excel_file_name)
                
                # Convert the text file to Excel
                txt_to_excel(file_path, excel_file_path)
                print(f"Converted {file_path} to {excel_file_path}\n")

print("Conversion completed.")


Converted C:\Users\nnngu\OneDrive\Documents\School\Georgia Tech\Summer 2024\K-12 Data Solutions\Data\NCES Data\NCES Demographic data\Parent of Relevant Children Demographic 2018-2022\PDP05_205_USSchoolDistrictAll_614125158923.txt to C:\Users\nnngu\OneDrive\Documents\School\Georgia Tech\Summer 2024\K-12 Data Solutions\Data\NCES Data\NCES Demographic data\Parent of Relevant Children Demographic 2018-2022\Parent of Relevant Children Demographic 2018-2022.xlsx

Converted C:\Users\nnngu\OneDrive\Documents\School\Georgia Tech\Summer 2024\K-12 Data Solutions\Data\NCES Data\NCES Demographic data\Parent of Relevant Children Economic 2018-2022\PDP03_205_USSchoolDistrictAll_614125110266.txt to C:\Users\nnngu\OneDrive\Documents\School\Georgia Tech\Summer 2024\K-12 Data Solutions\Data\NCES Data\NCES Demographic data\Parent of Relevant Children Economic 2018-2022\Parent of Relevant Children Economic 2018-2022.xlsx

Converted C:\Users\nnngu\OneDrive\Documents\School\Georgia Tech\Summer 2024\K-12 Data

In [2]:
# Get list of relevant columns from all excel files, keeping only keys, estimate and percent
# Excludes margin of error and percent margin of error columns
# Renaming the files starting with "Total children ..." and "Total Parents ..." because they have the same column names as those of some other files

import os
import re
import pandas as pd

# Get the current working directory (where the notebook is located)
current_dir = os.path.abspath("")

# Initialize an empty set to store unique column names
unique_columns = set()

# Columns to ensure are always included
always_included_columns = ['GeoId', 'Geography', 'LEAID', 'Year']
dont_rename = ['GeoId', 'Geography', 'LEAID', 'Year', 'Iteration']

# Function to add columns to unique_columns set
def add_columns_to_set(columns):
    for col in columns:
        unique_columns.add(col)

# Function to remove columns ending with "moe" or "pctmoe" and excluding 'Iteration'
def filter_columns(columns):
    filtered_columns = set()
    columns_to_remove = {'moe', 'pctmoe'}
    for col in columns:
        if not any(col.lower().endswith(suffix) for suffix in columns_to_remove) and col.lower() != 'iteration':
            filtered_columns.add(col)
    return filtered_columns

# Function to fix columns for files in "Total Children ..." and "Total Parents ..." folders
def fix_and_rename_columns(df, folder_name, excel_file_path):
    if folder_name.startswith("Total Children") or folder_name.startswith("Total Parents"):
        # Remove all existing "T" prefixes
        df.columns = [col.lstrip('T') if col not in dont_rename else col for col in df.columns]
        # Add "T" prefix back where necessary
        df.rename(columns={col: f'T{col}' for col in df.columns if col not in dont_rename}, inplace=True)
        print(f"Fixing and renaming columns in: {folder_name}")
        # Save the updated DataFrame back to the Excel file
        df.to_excel(excel_file_path, index=False)
    return df

# Add always included columns to the unique_columns set
add_columns_to_set(always_included_columns)

# Iterate over each folder in the current directory
for item_name in os.listdir(current_dir):
    item_path = os.path.join(current_dir, item_name)
    
    if os.path.isdir(item_path):
        # Construct the expected Excel file path
        excel_file_path = os.path.join(item_path, f"{item_name}.xlsx")
        
        # Check if the Excel file exists
        if os.path.exists(excel_file_path):
            print(f"Processing Excel file: {item_name}")
            
            # Read the data from the Excel file
            df = pd.read_excel(excel_file_path)
            
            # Fix and rename columns if in "Total Children ..." or "Total Parents ..." folders
            df = fix_and_rename_columns(df, item_name, excel_file_path)
            
            # Print the number of columns taken from this file before filtering
            print(f"Number of columns taken before filtering: {len(df.columns)}")

            # Filter columns based on unique_column_names
            filtered_cols = filter_columns(df.columns)
            print(f"Number of columns taken after filtering: {len(filtered_cols)}\n")

            # Add columns to unique_columns set
            add_columns_to_set(filtered_cols)

# Filter columns to remove those ending with "moe" or "pctmoe"
filtered_columns = filter_columns(unique_columns)

# Function to sort columns in the desired order
def sort_columns(cols):
    def custom_sort_key(col):
        match = re.match(r'(CDP|DP|PDP|TCDP|TPDP)(\d+)_(\d+)(est|pct)', col)
        if match:
            main_prefix, main_num, sub_num, suffix = match.groups()
            return (main_prefix, int(main_num), int(sub_num), suffix)
        return ('', float('inf'))

    return sorted(cols, key=custom_sort_key)

unique_column_names = sort_columns(filtered_columns)

# Ensure always included columns are at the beginning of the list
for col in reversed(always_included_columns):
    if col in unique_column_names:
        unique_column_names.remove(col)
    unique_column_names.insert(0, col)

# Print the count of unique column names
print(f"\nNumber of unique column names across all Excel files in current directory: {len(unique_column_names)}")

# Print or use the unique column names
print("\nUnique column names across all Excel files in current directory:")
for col in unique_column_names:
    print(col)



Processing Excel file: Parent of Relevant Children Demographic 2018-2022
Number of columns taken before filtering: 277
Number of columns taken after filtering: 140

Processing Excel file: Parent of Relevant Children Economic 2018-2022
Number of columns taken before filtering: 261
Number of columns taken after filtering: 132

Processing Excel file: Parent of Relevant Children Social 2018-2022
Number of columns taken before filtering: 435
Number of columns taken after filtering: 219

Processing Excel file: Relevant Children Demographic 2018-2022
Number of columns taken before filtering: 269
Number of columns taken after filtering: 136

Processing Excel file: Relevant Children Economic 2018-2022
Number of columns taken before filtering: 207
Number of columns taken after filtering: 105

Processing Excel file: Relevant Children Social 2018-2022
Number of columns taken before filtering: 353
Number of columns taken after filtering: 178

Processing Excel file: Total Children Demographic 2018-2

In [3]:
# Merging all 15 files into 1 with only selected columns (estimates and percentage)

import os
import pandas as pd

# Get the current working directory (where the script is located)
current_dir = os.path.abspath("")

# Initialize an empty DataFrame to store the combined data
demographic_main_data = pd.DataFrame(columns=unique_column_names)

# Initialize counters for tracking changes
total_columns_filled = 0
total_rows_added = 0

# Set to keep track of unique columns filled across all files
columns_filled_set = set()

# Iterate over each folder in the current directory
for item_name in os.listdir(current_dir):
    item_path = os.path.join(current_dir, item_name)
    
    if os.path.isdir(item_path):
        # Construct the expected Excel file path
        excel_file_path = os.path.join(item_path, f"{item_name}.xlsx")
        
        # Check if the Excel file exists
        if os.path.exists(excel_file_path):
            print(f"Processing Excel file: {item_name}")
            
            # Read the data from the Excel file
            df = pd.read_excel(excel_file_path)
            
            # Filter columns based on unique_column_names
            df_filtered = df[[col for col in df.columns if col in unique_column_names]]

            # Merge the filtered data frame into the main demographic data frame
            demographic_main_data = pd.merge(
                demographic_main_data,
                df_filtered,
                on=['GeoId', 'Geography', 'LEAID', 'Year'],
                how='outer',
                suffixes=('', '_new')
            )

            # Update the columns filled with new data
            for col in df_filtered.columns:
                if col not in ['GeoId', 'Geography', 'LEAID', 'Year']:
                    # If the original column contains NaN, fill it with the new data
                    demographic_main_data[col].fillna(demographic_main_data[f'{col}_new'], inplace=True)
                    # Drop the new column
                    demographic_main_data.drop(columns=[f'{col}_new'], inplace=True)
                    # Add to the set of filled columns
                    columns_filled_set.add(col)

            # Log changes after processing the data frame
            num_rows_after = len(demographic_main_data)
            num_new_rows = num_rows_after - total_rows_added
            total_rows_added += num_new_rows
#             total_columns_filled += len(df_filtered.columns) - 4  # Exclude LEAID, Geography, GeoId, Year
            print(f"    Rows added: {num_new_rows}")
            print(f"    Columns filled with data: {len(columns_filled_set)}")
            print(f"    Total rows in combined data: {num_rows_after}")
            print(f"    Total columns in combined data: {len(demographic_main_data.columns)}\n")

            
# Save the combined DataFrame to a new Excel file
output_file_path = os.path.abspath('Demographic_main_data.xlsx')
demographic_main_data.to_excel(output_file_path, index=False)

print(f"\nData from all Excel files has been combined and saved to: {output_file_path}")
print(f"Total rows: {total_rows_added}")
print(f"Total columns: {len(demographic_main_data.columns)}")


Processing Excel file: Parent of Relevant Children Demographic 2018-2022
    Rows added: 10683
    Columns filled with data: 136
    Total rows in combined data: 10683
    Total columns in combined data: 2442

Processing Excel file: Parent of Relevant Children Economic 2018-2022
    Rows added: 0
    Columns filled with data: 264
    Total rows in combined data: 10683
    Total columns in combined data: 2442

Processing Excel file: Parent of Relevant Children Social 2018-2022
    Rows added: 0
    Columns filled with data: 479
    Total rows in combined data: 10683
    Total columns in combined data: 2442

Processing Excel file: Relevant Children Demographic 2018-2022
    Rows added: 705
    Columns filled with data: 611
    Total rows in combined data: 11388
    Total columns in combined data: 2442

Processing Excel file: Relevant Children Economic 2018-2022
    Rows added: 0
    Columns filled with data: 712
    Total rows in combined data: 11388
    Total columns in combined data: 2

In [4]:
# Reorder the columns to ensure always_included_columns are first
ordered_columns = always_included_columns + [col for col in demographic_main_data.columns if col not in always_included_columns]
demographic_main_data = demographic_main_data[ordered_columns]

# Save the combined DataFrame to a new Excel file
output_file_path = os.path.abspath('Demographic_main_data.xlsx')
demographic_main_data.to_excel(output_file_path, index=False)

print(f"\nData from all Excel files has been combined and saved to: {output_file_path}")
print(f"Total rows added: {total_rows_added}")
print(f"Total columns filled with data: {len(demographic_main_data.columns)}")


Data from all Excel files has been combined and saved to: C:\Users\nnngu\OneDrive\Documents\School\Georgia Tech\Summer 2024\K-12 Data Solutions\Data\NCES Data\NCES Demographic data\Demographic_main_data.xlsx
Total rows added: 13399
Total columns filled with data: 2442


In [5]:
# Splitting the main data into 2: one for estimates and one for percentage

# Columns to retain in both datasets
common_columns = ['GeoId', 'Geography', 'LEAID', 'Year']

# Identify columns that end with "est" and "pct"
est_columns = [col for col in demographic_main_data.columns if col.endswith('est')]
pct_columns = [col for col in demographic_main_data.columns if col.endswith('pct')]

# Create datasets
df_est = demographic_main_data[common_columns + est_columns]
df_pct = demographic_main_data[common_columns + pct_columns]

# Save the new datasets to Excel files
df_est.to_excel('Demographic_main_data_est.xlsx', index=False)
df_pct.to_excel('Demographic_main_data_pct.xlsx', index=False)

# Display the new datasets (optional)
print("EST Dataset:\n", df_est.head())
print("PCT Dataset:\n", df_pct.head())


EST Dataset:
               GeoId                                   Geography   LEAID  \
0  9700000US0200007  Aleutians East Borough School District, AK  200007   
1  9700000US0200180               Anchorage School District, AK  200180   
2  9700000US0200020           Bering Strait School District, AK  200020   
3  9700000US0200030     Bristol Bay Borough School District, AK  200030   
4  9700000US0200070            Copper River School District, AK  200070   

        Year  CDP02_1est  CDP02_2est  CDP02_3est  CDP02_4est  CDP02_5est  \
0  2018-2022       215.0       100.0        45.0        15.0        55.0   
1  2018-2022     23970.0     15390.0      2300.0      2030.0      4250.0   
2  2018-2022       935.0       380.0       205.0       145.0       210.0   
3  2018-2022        40.0        30.0         4.0         4.0         4.0   
4  2018-2022       170.0       120.0         4.0        25.0        20.0   

   CDP02_6est  ...  TPDP05_62est  TPDP05_63est  TPDP05_64est  TPDP05_65est  \
